In [ ]:
!pip install openai tiktoken langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import openai
import numpy as np
import pandas as pd

from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import get_embedding, cosine_similarity
from sklearn.cluster import KMeans

In [ ]:
openai.api_key = ''

def split_text_into_chunks(text, chunk_size):
    chunks = []
    words = text.split()
    current_chunk = ''

    for word in words:
        if len(current_chunk) + len(word) + 1 <= chunk_size:
            current_chunk += word + ' '
        else:
            chunks.append(current_chunk.strip())
            current_chunk = word + ' '

    chunks.append(current_chunk.strip())
    return chunks

def extract_major_ideas(text):
    major_ideas = []

    max_iterations = 10
    chunks = split_text_into_chunks(text, 4000)
    conversation = [{'role': 'system', 'content': 'You are a helpful assistant that extracts key points from text.'}]
    for chunk in chunks:
        conversation.append({'role': 'user', 'content': chunk})
        response = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=conversation,
            max_tokens=1000,
        )
        user_message = response.choices[0].message['content']
        major_ideas.append(user_message)
        conversation = [{'role': 'system', 'content': 'You are a helpful assistant that extracts key points from text.'}]
    return major_ideas

In [ ]:
with open('/content/drive/MyDrive/calhacks/article.txt', 'r') as file:
    large_text = file.read()

In [ ]:
major_ideas = extract_major_ideas(large_text)

In [ ]:
all_major_ideas = []
for major_idea in major_ideas:
  for keypoint in major_idea.split("\n"):
    all_major_ideas.append(keypoint[2:])
all_major_ideas

['Autonomous Driving (AD) systems rely on AI for safety and driving decisions, but these are vulnerable to adversarial attacks.',
 '"Semantic AI security" is the term referred to the research that explores the implications of these attacks not just at the component level, but at the system-wide level.',
 'Over the past five years, research in this field has grown and is now being performed to address these challenges in AD.',
 'This paper aims to systematically organise this emerging research field by analysing 53 papers. It categorises them based on aspects like the AI component targeted, the goal of the attack or defence, attack vectors, defence robustness, and evaluation methodologies.',
 'Six significant scientific gaps are identified and the paper provides potential future directions for research and design.',
 'An open source evaluation platform called PASS has been developed to address the gap in scientific methodology.',
 'While AI components are vulnerable to attacks, these do

In [ ]:
response = openai.Embedding.create(
    input=major_ideas,
    model="text-similarity-babbage-001"
)

In [ ]:
corpus_embeddings = [ d['embedding'] for d in response['data']]
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

In [ ]:
clustering_model = KMeans(n_clusters=10)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[8 2 8 1 2 9 7 5 4 3 0 0 0 1 0 6]


In [ ]:
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(major_ideas[sentence_id])
clustered_sentences

{8: ['- Autonomous Driving (AD) systems rely on AI for safety and driving decisions, but these are vulnerable to adversarial attacks.\n- "Semantic AI security" is the term referred to the research that explores the implications of these attacks not just at the component level, but at the system-wide level.\n- Over the past five years, research in this field has grown and is now being performed to address these challenges in AD.\n- This paper aims to systematically organise this emerging research field by analysing 53 papers. It categorises them based on aspects like the AI component targeted, the goal of the attack or defence, attack vectors, defence robustness, and evaluation methodologies.\n- Six significant scientific gaps are identified and the paper provides potential future directions for research and design.\n- An open source evaluation platform called PASS has been developed to address the gap in scientific methodology.\n- While AI components are vulnerable to attacks, these do

In [ ]:
clustered_sentences

{8: ['- Autonomous Driving (AD) systems rely on AI for safety and driving decisions, but these are vulnerable to adversarial attacks.\n- "Semantic AI security" is the term referred to the research that explores the implications of these attacks not just at the component level, but at the system-wide level.\n- Over the past five years, research in this field has grown and is now being performed to address these challenges in AD.\n- This paper aims to systematically organise this emerging research field by analysing 53 papers. It categorises them based on aspects like the AI component targeted, the goal of the attack or defence, attack vectors, defence robustness, and evaluation methodologies.\n- Six significant scientific gaps are identified and the paper provides potential future directions for research and design.\n- An open source evaluation platform called PASS has been developed to address the gap in scientific methodology.\n- While AI components are vulnerable to attacks, these do

In [ ]:
clusterKeyToTheme = {}
for cluster_key in clustered_sentences.keys():

  allsentencesincluster = ' '.join(clustered_sentences[cluster_key])
  response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f'What do the following key points have in common?\n\nKey points:\n"""\n{allsentencesincluster}\n"""\n\nTheme:',
        temperature=0,
        max_tokens=64,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
  print('Cluster num', cluster_key, response["choices"][0]["text"].replace("\n", ""))
  clusterKeyToTheme[cluster_key] = response["choices"][0]["text"].replace("\n", "")

Cluster num 8  Autonomous Driving (AD) AI security research and the development of an open source evaluation platform called PASS to address the gap in scientific methodology.
Cluster num 2  The authors analyze research papers related to semantic AI security in the context of autonomous driving, and identify scientific gaps and potential future directions.
Cluster num 1  The security of Autonomous Driving (AD) AI stack and the need to study the security properties of downstream AI components.
Cluster num 9  Autonomous Driving AI Research and Security
Cluster num 7  Attacks on Autonomous Driving Systems
Cluster num 5 The common theme among the key points is the various methods of hacking autonomous driving (AD) systems.
Cluster num 4  Autonomous Driving (AD) AI Systems
Cluster num 3  Improving the robustness of AI components against attacks through defense methods.
Cluster num 0  The lack of system-level evaluation and defense solutions in Autonomous Driving (AD) AI security research, a

In [ ]:
for cluster_key in clustered_sentences.keys():
  print(cluster_key)
  print(clusterKeyToTheme[cluster_key])
  print(clustered_sentences[cluster_key])

8
 Autonomous Driving (AD) AI security research and the development of an open source evaluation platform called PASS to address the gap in scientific methodology.
['- Autonomous Driving (AD) systems rely on AI for safety and driving decisions, but these are vulnerable to adversarial attacks.\n- "Semantic AI security" is the term referred to the research that explores the implications of these attacks not just at the component level, but at the system-wide level.\n- Over the past five years, research in this field has grown and is now being performed to address these challenges in AD.\n- This paper aims to systematically organise this emerging research field by analysing 53 papers. It categorises them based on aspects like the AI component targeted, the goal of the attack or defence, attack vectors, defence robustness, and evaluation methodologies.\n- Six significant scientific gaps are identified and the paper provides potential future directions for research and design.\n- An open so

In [ ]:
clusterKeyToTheme.values()

dict_values([' Autonomous Driving (AD) AI security research and the development of an open source evaluation platform called PASS to address the gap in scientific methodology.', ' The authors analyze research papers related to semantic AI security in the context of autonomous driving, and identify scientific gaps and potential future directions.', ' The security of Autonomous Driving (AD) AI stack and the need to study the security properties of downstream AI components.', ' Autonomous Driving AI Research and Security', ' Attacks on Autonomous Driving Systems', 'The common theme among the key points is the various methods of hacking autonomous driving (AD) systems.', ' Autonomous Driving (AD) AI Systems', ' Improving the robustness of AI components against attacks through defense methods.', ' The lack of system-level evaluation and defense solutions in Autonomous Driving (AD) AI security research, and the need for more research in this area.', ' The benefits of open-sourcing efforts wi

In [ ]:
clusterKeyToTheme

{8: ' Autonomous Driving (AD) AI security research and the development of an open source evaluation platform called PASS to address the gap in scientific methodology.',
 2: ' The authors analyze research papers related to semantic AI security in the context of autonomous driving, and identify scientific gaps and potential future directions.',
 1: ' The security of Autonomous Driving (AD) AI stack and the need to study the security properties of downstream AI components.',
 9: ' Autonomous Driving AI Research and Security',
 7: ' Attacks on Autonomous Driving Systems',
 5: 'The common theme among the key points is the various methods of hacking autonomous driving (AD) systems.',
 4: ' Autonomous Driving (AD) AI Systems',
 3: ' Improving the robustness of AI components against attacks through defense methods.',
 0: ' The lack of system-level evaluation and defense solutions in Autonomous Driving (AD) AI security research, and the need for more research in this area.',
 6: ' The benefits 